In [69]:
import re

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch._tensor import Tensor
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence

import os
import pandas as pd
import math

# Deep Learning for NLP - Lab Assignment 2

## Loading Data


In [70]:
DATA_PATH = "../data/google" # Folder path for the train/test/dev .tsv files

train_data_raw = pd.read_csv(os.path.join(DATA_PATH, "train.tsv"), sep='\t', header=None)[0].tolist()[:5000] # Necessary to take a subset of the dataset due to computation constraints
test_data_raw = pd.read_csv(os.path.join(DATA_PATH, "test.tsv"), sep='\t', header=None)[0].tolist()[:1000]
dev_data_raw = pd.read_csv(os.path.join(DATA_PATH, "dev.tsv"), sep='\t', header=None)[0].tolist()[:1000]

In [71]:
def clean_str(string: str, tolower:bool=True) -> str:
    """
    Tokenization/string cleaning.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " ( ", string)
    string = re.sub(r"\)", " ) ", string)
    string = re.sub(r"\?", " ? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    if tolower:
        string = string.lower()
    return string.strip()


train_data, test_data, dev_data = [], [], []

for sentence in train_data_raw:
    train_data.append(clean_str(sentence))
for sentence in dev_data_raw:
    dev_data.append(clean_str(sentence))
for sentence in test_data_raw:
    test_data.append(clean_str(sentence))

## Creating vocabulary


In [72]:
class WordDict:
    """Word dictionnary class.
    """
    # constructor, words must be a set containing all words
    def __init__(self, words:set) -> None:
        """Initialize a word dictionnary

        Args:
            words (set): set of all words in a dataset
        """
        assert type(words) == set
        self.word_to_idx = {word: idx for idx, word in enumerate(words)}
        self.idx_to_word = {idx: word for word, idx in self.word_to_idx.items()}

    def word_to_id(self, word:str) -> int:
        """Return the integer associated with a word.

        Args:
            word (str): word

        Returns:
            int: index of the word in the vocabulary
        """
        return self.word_to_idx[word]
    
    def id_to_word(self, idx:int) -> str:
        """Return the word associated with an integer.

        Args:
            idx (int): integer

        Returns:
            str: word at that index in the word dictionnary
        """
        return self.idx_to_word[idx]
    
    def __len__(self) -> int:
        """Compute length of the dictionnary

        Returns:
            int: length of the dictionnary of words
        """
        return len(self.word_to_idx)

In [73]:
train_words = set()

for sentence in train_data:
    train_words.update(sentence.split(" "))

train_words.update(["<bos>", "<eos>", "<unk>", "<pad>"])

word_dict = WordDict(train_words)

print("Number of words :", len(word_dict))
print(list(word_dict.idx_to_word.items())[0:5]) # Excerpt of the idx_to_word dictionnary

Number of words : 8159
[(0, 'knee'), (1, 'kittens'), (2, 'cmmon'), (3, 'likely'), (4, 'listen')]


## Neural N-Gram model


In [74]:
class NGramDataset(Dataset):
    """Dataset for training the N-gram model, based on the Pytorch Dataset class.
    """
    def __init__(self, sentences:list[str], vocab:WordDict, context_size:int) -> None:
        """N-gram Dataset to use with the DataLoader feature of torch during training

        Args:
            sentences (list[str]): List of sentences
            vocab (WordDict): _description_
            context_size (int): _description_
        """
        super().__init__()
        self.data = []
        self.vocab = vocab
        self.context_size = context_size

        for sentence in sentences:
            tokens = ["<bos>"] * context_size + sentence.split(" ") + ["<eos>"] # Pad the sentence and extract words. We pad the sentence with context_size <bos> tokens for generating the first word.
            indices = [vocab.word_to_id(word) if word in vocab.word_to_idx.keys() else vocab.word_to_id('<unk>') for word in tokens] # Tokenize the sentence using the WordDict
            for i in range(context_size, len(indices)):
                self.data.append((torch.tensor(indices[i-context_size:i]),
                                              torch.tensor(indices[i]))) # Append the training data with a tuple ([word_1, word_2 ,...], word_n)

    def __len__(self) -> int:
        """Compute len of dataset (necessary)

        Returns:
            int: length of the dataset
        """
        return len(self.data)
    
    def __getitem__(self, idx:int) -> tuple[Tensor, Tensor]:
        """Return a tuple of tensor, containing encodings of the n-grams and next word to predict

        Args:
            idx (int): index of the item in the dataset

        Returns:
            tuple[Tensor, Tensor]: Tuple with n-gram data and next word in the sentence
        """
        return self.data[idx]

In [75]:
# Define the context size (e.g., 2 for bigrams)
CONTEXT_SIZE = 5  # For n-grams, context_size = n - 1

# Dataset
ngram_train_dataset = NGramDataset(train_data, word_dict, CONTEXT_SIZE)
ngram_dev_dataset = NGramDataset(dev_data, word_dict, CONTEXT_SIZE)
ngram_test_dataset = NGramDataset(test_data, word_dict, CONTEXT_SIZE)

print("Train_dataset length :", len(ngram_train_dataset))
print("Training item example :", ngram_train_dataset[123]) # Example of item in the training data
print("Training sentence example :", " ".join([word_dict.id_to_word(id) for id in ngram_train_dataset[123][0].tolist()]), "; Target word example :", word_dict.id_to_word(ngram_train_dataset[123][1].item()))

Train_dataset length : 74083
Training item example : (tensor([7424, 1247, 2858, 5271, 2334]), tensor(4709))
Training sentence example : youtube and outrage drama is ; Target word example : super


In [76]:
class NeuralNGramModel(nn.Module):
    """Class for the Neural N-gram module, based on the Pytorch based class.
    """
    def __init__(self, vocab_size:int, embed_size:int, context_size:int, hidden_size:int) -> None:
        """Initialize the Neural N-gram model

        Args:
            vocab_size (int): Number of words in the dictionnary
            embed_size (int): Embedding size
            context_size (int): Number 
            hidden_size (int): _description_
        """
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_size)
        self.fc1 = nn.Linear(embed_size * context_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x:Tensor) -> Tensor:
        """Forward pass of the model

        Args:
            x (Tensor): input of the model (batched), size (batch_size, context_size)

        Returns:
            Tensor: output of the model, in the form of a (vocab_size, x.shape(1)) tensor
        """

        embeds = self.embeddings(x) # (batch_size, context_size, embed_size)
        embeds = embeds.view(embeds.size(0), -1) # (batch_size, embed_size * context_size)

        hidden = self.dropout(F.relu(self.fc1(embeds))) # (batch_size, hidden_size)
        output = self.fc2(hidden) # (batch_size, vocab_size)
        
        return output


In [77]:
# Model and training parameters
NGRAM_EMBED_SIZE = 16
NGRAM_HIDDEN_SIZE = 16
NGRAM_BATCH_SIZE = 32
NGRAM_EPOCHS = 10

# DataLoader
ngram_train_loader = DataLoader(ngram_train_dataset, batch_size=NGRAM_BATCH_SIZE, shuffle=True)
ngram_dev_loader = DataLoader(ngram_dev_dataset, batch_size=NGRAM_BATCH_SIZE)

# Initialize Model, Loss, Optimizer
vocab_size = len(word_dict)
ngram_model = NeuralNGramModel(vocab_size, NGRAM_EMBED_SIZE, CONTEXT_SIZE, NGRAM_HIDDEN_SIZE)
ngram_criterion = nn.CrossEntropyLoss(ignore_index=word_dict.word_to_id('<pad>')) # We can see the problem as a multi-label classification problem, classyfying among all possibles words in the vocab
ngram_optimizer = optim.Adam(ngram_model.parameters())

# Training Loop

for epoch in range(NGRAM_EPOCHS):
    ngram_model.train()
    ngram_train_loss = 0

    # Train loop
    for context, target in ngram_train_loader:

        # Forward pass
        output = ngram_model(context)
        loss = ngram_criterion(output, target)

        # Backpropagation
        ngram_optimizer.zero_grad()
        loss.backward()
        ngram_optimizer.step()

        ngram_train_loss += loss.item()
    
    # Dev loop
    ngram_model.eval()
    ngram_dev_loss = 0
    with torch.no_grad():
        for context, target in ngram_dev_loader:

            # Forward pass
            output = ngram_model(context)
            loss = ngram_criterion(output, target)
            ngram_dev_loss += loss.item()

    ngram_train_loss /= len(ngram_train_loader)
    ngram_dev_loss /= len(ngram_dev_loader)

    print(f"Epoch [{epoch+1}/{NGRAM_EPOCHS}], "
            f"Train Loss: {ngram_train_loss:.2f}, "
            f"Dev Loss: {ngram_dev_loss:.2f}")

Epoch [1/10], Train Loss: 6.88, Dev Loss: 6.47
Epoch [2/10], Train Loss: 6.39, Dev Loss: 6.49
Epoch [3/10], Train Loss: 6.27, Dev Loss: 6.50
Epoch [4/10], Train Loss: 6.16, Dev Loss: 6.59
Epoch [5/10], Train Loss: 6.07, Dev Loss: 6.65
Epoch [6/10], Train Loss: 5.99, Dev Loss: 6.74
Epoch [7/10], Train Loss: 5.91, Dev Loss: 6.82
Epoch [8/10], Train Loss: 5.85, Dev Loss: 6.93
Epoch [9/10], Train Loss: 5.79, Dev Loss: 7.03
Epoch [10/10], Train Loss: 5.74, Dev Loss: 7.14


## Sentence Generation


In [78]:
# Generation example

ngram_model.eval()

context = ["<bos>"] * CONTEXT_SIZE


generated_sequence = context[:]
while generated_sequence[-1] != "<eos>":
    context_indices = torch.tensor([word_dict.word_to_id(word) for word in generated_sequence[-CONTEXT_SIZE:]], dtype = torch.long).unsqueeze(0)
    with torch.no_grad():
        output = ngram_model(context_indices)
        probabilities = torch.softmax(output, dim=-1)

    predicted_index = torch.argmax(probabilities, dim=-1).item()
    predicted_word = word_dict.id_to_word(predicted_index)
    generated_sequence.append(predicted_word)

print("Deterministic generated sentence :", " ".join(generated_sequence[CONTEXT_SIZE:-1]))

Deterministic generated sentence : i do n't a the


In [79]:
# Generation example

ngram_model.eval()

context = ["<bos>"] * CONTEXT_SIZE
context_indices = torch.tensor([word_dict.word_to_id(word) for word in context], dtype = torch.long).unsqueeze(0)

for _ in range(10):
    generated_sequence = context[:]
    while generated_sequence[-1] != "<eos>":
        context_indices = torch.tensor([word_dict.word_to_id(word) for word in generated_sequence[-CONTEXT_SIZE:]], dtype = torch.long).unsqueeze(0)
        with torch.no_grad():
            output = ngram_model(context_indices)
            probabilities = torch.softmax(output, dim=-1)

        predicted_index = torch.multinomial(probabilities.squeeze(), num_samples=1).item()
        predicted_word = word_dict.id_to_word(predicted_index)
        generated_sequence.append(predicted_word)

    print("Random generated sequence :", " ".join(generated_sequence[CONTEXT_SIZE:-1]))

Random generated sequence : so yeah , then ?
Random generated sequence : certainly guy fucking


Random generated sequence : me little only unbelievable face it 's sucks under him i for you never someone for looking to episodes
Random generated sequence : this me the screen ? there is the gonna or just cover the family is very even i make the in world why the focus
Random generated sequence : please first this pm , name s a these goshdarn people that is people
Random generated sequence : , what died i mean
Random generated sequence : lol it that appreciate see 's conversation look if you those the life for being disgusting there t d in
Random generated sequence : i ca n't family on faith sorry to bud
Random generated sequence : it 's did have be are worse post say on picture
Random generated sequence : we fucking sure for day are the favourite it , terrible more then her bummer


## LSTM-based Autoregressive Model


In [80]:
class LSTMDataset(Dataset):
    """Dataset for training the LSTM model, based on the Pytorch Dataset class.
    """
    def __init__(self, sentences:list[str], vocab:WordDict) -> None:
        """_summary_

        Args:
            sentences (list[str]): List od sentences.
            vocab (WordDict): _description_
        """
        super().__init__()
        self.data = []
        self.vocab = vocab

        for sentence in sentences:
            tokens = ["<bos>"] + sentence.split(" ") + ["<eos>"]
            indices = [vocab.word_to_id(word) if word in vocab.word_to_idx.keys() else vocab.word_to_id('<unk>') for word in tokens] # Tokenize the sentence using the WordDict
            
            self.data.append((torch.tensor(indices),
                              torch.tensor(indices[1:]+[self.vocab.word_to_id('<pad>')])))
            
    def __len__(self) -> int:
        """Compute len of dataset (necessary)

        Returns:
            int: length of the dataset
        """
        return len(self.data)
    
    def __getitem__(self, idx:int) -> tuple[Tensor, Tensor]:
        """Return a tuple of tensor, containing encodings of the n-grams and next word to predict

        Args:
            idx (int): index of the item in the dataset

        Returns:
            tuple[Tensor, Tensor]: Tuple with sentence and shifted sentence for generation
        """
        return self.data[idx]


def collate_fn(batch:list[tuple]) -> tuple[Tensor, Tensor]:
    """Custom collate function for dynamically padding sentences

    Args:
        batch (list[tuple]): batch of a sentence Dataset

    Returns:
        tuple[Tensor, Tensor]: Dynamicallt padded sentences
    """
    sentences, shifted_sentences = zip(*batch)

    padded_sentences = pad_sequence(sentences, batch_first=True, padding_value=word_dict.word_to_id('<pad>'))
    padded_shifted_sentences = pad_sequence(shifted_sentences, batch_first=True, padding_value=word_dict.word_to_id('<pad>'))

    return padded_sentences, padded_shifted_sentences

In [81]:
# Dataset
lstm_train_dataset = LSTMDataset(train_data, word_dict)
lstm_dev_dataset = LSTMDataset(dev_data, word_dict)

print("Train_dataset length :", len(lstm_train_dataset))
print("Training item example :", lstm_train_dataset[123]) # Example of item in the training data
print("Training sentence example :", " ".join([word_dict.id_to_word(id) for id in lstm_train_dataset[123][0].tolist()]), " ; Target sentence example :", " ".join([word_dict.id_to_word(id) for id in lstm_train_dataset[123][1].tolist()]))

Train_dataset length : 5000
Training item example : (tensor([1448, 4045, 4330, 7788, 7605,  470, 1886, 7421, 4973, 6016]), tensor([4045, 4330, 7788, 7605,  470, 1886, 7421, 4973, 6016, 7720]))
Training sentence example : <bos> three words , no subtlety dude stop seriously <eos>  ; Target sentence example : three words , no subtlety dude stop seriously <eos> <pad>


In [82]:
class LSTMModel(nn.Module):
    """Class for the LSTM module, based on the Pytorch based class.
    """
    def __init__(self, vocab_size:int, embed_size:int, hidden_size:int, dropout_prob:float=0.3) -> None:
        """Initialize the LSTM

        Args:
            vocab_size (int): Number of words in the dictionnary
            embed_size (int): Embedding size
            hidden_size (int): Size of the hidden layer
        """
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

        self.dropout_prob = dropout_prob

    def variational_dropout(self, input:Tensor) -> Tensor:
        if self.training: # Apply variational dropout only during training
            mask = (torch.rand_like(input) > self.dropout_prob).float().to(input.device)
            mask = mask.div_(1.0 - self.dropout_prob)
            return input * mask
        return input # If not in training, return input

    def forward(self, x:Tensor) -> Tensor:
        """Forward pass of the model

        Args:
            x (Tensor): input of the model (batched), size (batch_size, context_size)

        Returns:
            Tensor: output of the model, in the form of a (vocab_size, x.shape(1)) tensor
        """

        embeds = self.embeddings(x) # (batch_size, seq_len, embed_size)
        embeds = self.variational_dropout(embeds) # (batch_size, seq_len, embed_size)

        output, _ = self.lstm(embeds) # (batch_size, seq_len, hidden_size)
        output = self.variational_dropout(output) # (batch_size, seq_len, hidden_size)

        output = self.fc(output) # (batch_size, seq_len, vocab_size)

        
        return output


In [ ]:
# Model and training parameters
LSTM_EMBED_SIZE = 16
LSTM_HIDDEN_SIZE = 16
LSTM_BATCH_SIZE = 32
LSTM_EPOCHS = 10

# DataLoader
lstm_train_loader = DataLoader(lstm_train_dataset, batch_size=LSTM_BATCH_SIZE, collate_fn=collate_fn, shuffle=True)
lstm_dev_loader = DataLoader(lstm_dev_dataset, batch_size=LSTM_BATCH_SIZE, collate_fn=collate_fn)

# Initialize Model, Loss, Optimizer
vocab_size = len(word_dict)
lstm_model = LSTMModel(vocab_size, LSTM_EMBED_SIZE, LSTM_HIDDEN_SIZE)
lstm_criterion = nn.CrossEntropyLoss(ignore_index=word_dict.word_to_id('<pad>')) # We can see the problem as a multi-label classification problem, classyfying among all possibles words in the vocab
lstm_optimizer = optim.Adam(lstm_model.parameters())

# Training Loop

for epoch in range(LSTM_EPOCHS):
    lstm_model.train()
    lstm_train_loss = 0

    # Train loop
    for context, target in lstm_train_loader:

        # Forward pass
        output = lstm_model(context)

        # Reshape for loss calculation
        output = output.view(-1, vocab_size) # (batch_size *  seq_len, vocab_size)
        target = target.view(-1) # (batch_size * seq_len)

        # Compute loss
        loss = lstm_criterion(output, target)

        # Backpropagation
        lstm_optimizer.zero_grad()
        loss.backward()
        lstm_optimizer.step()

        lstm_train_loss += loss.item()
    
    # Dev loop
    lstm_model.eval()
    lstm_dev_loss = 0
    with torch.no_grad():
        for context, target in lstm_dev_loader:

            # Forward pass
            output = lstm_model(context)

            # Reshape for loss calculation
            output = output.view(-1, vocab_size) # (batch_size *  seq_len, vocab_size)
            target = target.view(-1) # (batch_size * seq_len)

            # Compute loss
            loss = lstm_criterion(output, target)

            lstm_dev_loss += loss.item()

    lstm_train_loss /= len(lstm_train_loader)
    lstm_dev_loss /= len(lstm_dev_loader)

    print(f"Epoch [{epoch+1}/{LSTM_EPOCHS}], "
            f"Train Loss: {lstm_train_loss:.2f}, "
            f"Dev Loss: {lstm_dev_loss:.2f}")

Epoch [1/10], Train Loss: 8.30, Dev Loss: 0.00
Epoch [2/10], Train Loss: 6.65, Dev Loss: 0.00
Epoch [3/10], Train Loss: 6.53, Dev Loss: 0.00
Epoch [4/10], Train Loss: 6.50, Dev Loss: 0.00
Epoch [5/10], Train Loss: 6.47, Dev Loss: 0.00
Epoch [6/10], Train Loss: 6.45, Dev Loss: 0.00
Epoch [7/10], Train Loss: 6.44, Dev Loss: 0.00
Epoch [8/10], Train Loss: 6.42, Dev Loss: 0.00
Epoch [9/10], Train Loss: 6.40, Dev Loss: 0.00
Epoch [10/10], Train Loss: 6.38, Dev Loss: 0.00


## Sentence Generation


In [ ]:
# Generation example

lstm_model.eval()

context = ["<bos>"]
context_indices = torch.tensor([word_dict.word_to_id(word) for word in context], dtype = torch.long).unsqueeze(0)

MAX_LEN = 100

generated_sequence = context[:]
while generated_sequence[-1] != "<eos>" and len(generated_sequence) < MAX_LEN:
    context_indices = torch.tensor([word_dict.word_to_id(word) for word in generated_sequence], dtype = torch.long).unsqueeze(0)
    with torch.no_grad():
        output = lstm_model(context_indices)[:, -1]
        probabilities = torch.softmax(output, dim=-1)

    predicted_index = torch.argmax(probabilities, dim=-1).item()
    predicted_word = word_dict.id_to_word(predicted_index)
    generated_sequence.append(predicted_word)

print("Deterministic generated sentence :", " ".join(generated_sequence[1:-1]))

Deterministic generated sentence : <bos> <eos>


In [85]:
# Generation example

ngram_model.eval()

context = ["<bos>"]
context_indices = torch.tensor([word_dict.word_to_id(word) for word in context], dtype = torch.long).unsqueeze(0)

MAX_LEN = 100

for _ in range(10):
    generated_sequence = context[:]
    while generated_sequence[-1] != "<eos>" and len(generated_sequence) < MAX_LEN:
        context_indices = torch.tensor([word_dict.word_to_id(word) for word in generated_sequence], dtype = torch.long).unsqueeze(0)
        with torch.no_grad():
            output = lstm_model(context_indices)[:, -1]
            probabilities = torch.softmax(output, dim=-1)

        predicted_index = torch.multinomial(probabilities.squeeze(), num_samples=1).item()
        predicted_word = word_dict.id_to_word(predicted_index)
        generated_sequence.append(predicted_word)

    print("Random generated sentence :", " ".join(generated_sequence[1:-1]))

Random generated sentence : argument
Random generated sentence : your fact horses been good that 1 sorry nice in adjustments manila to shockingly of like that to fucking is this bitcoin jersey want feedback of though gonna watching get ? she
Random generated sentence : them i to flag is nostrils double bro think my 180 mate results , be ! ! it dude be say a she in the lma0 doods you anyone an fail
Random generated sentence : i the its to play wasting i'm all her slang an tweet below seconds stop bb ! not and my exploiting so can i little with to
Random generated sentence : fuck to game you would
Random generated sentence : when of of five she can love that
Random generated sentence : hard of yay still n't if 6 great my human as peach completely hope i'm the edited troo s stability
Random generated sentence : this keep bad really blackhawks addition started your faster the friends but is like i think to be , but all help wonder was for lineup still envy , most again my 'll the essential

## Perplexity


In [86]:
class Perplexity:
    """Perplexity computation
    """
    def __init__(self) -> None:
        """Init method.
        """
        self.reset()
    def reset(self) -> None:
        """Rset method.
        """
        self.log_sum = 0
        self.total_words = 0

    def add_sentence(self, log_probs) -> None:
        """Compute values for one sentence and store it in the class.
        """
        self.log_sum += log_probs.sum().item()
        self.total_words += log_probs.size(0)

    def compute_perplexity(self) -> float:
        """Compute full Perplexity

        Returns:
            float: Final perplexity
        """
        return math.exp(-self.log_sum / self.total_words)

In [87]:
ngram_model.eval()
perplexity_object = Perplexity()

# Dataset & DataLoader
ngram_test_dataset = NGramDataset(test_data, word_dict, CONTEXT_SIZE)
ngram_test_loader = DataLoader(ngram_test_dataset, batch_size=NGRAM_BATCH_SIZE)

with torch.no_grad():
    for context, target in ngram_test_loader:
        output = ngram_model(context)
        log_probs = torch.log_softmax(output, dim=1)
        for i in range(len(target)):
            perplexity_object.add_sentence(log_probs[i][target[i]].unsqueeze(0))

perplexity = perplexity_object.compute_perplexity()

print(f"N-Gram Model Perplexity : {perplexity:.2f}")

N-Gram Model Perplexity : 1400.87


In [88]:
lstm_model.eval()
perplexity_object.reset()

# Dataset & DataLoader
lstm_test_dataset = LSTMDataset(test_data, word_dict)
lstm_test_loader = DataLoader(lstm_test_dataset, batch_size=LSTM_BATCH_SIZE, collate_fn=collate_fn)

with torch.no_grad():
    for context, target in lstm_test_loader:
        output = lstm_model(context)
        log_probs = torch.log_softmax(output, dim=2)

        log_probs = log_probs.view(-1, vocab_size)
        target = target.view(-1)

        mask = (target != word_dict.word_to_id('<pad>'))
        log_probs = log_probs[torch.arange(len(target))[mask], target[mask]]

        perplexity_object.add_sentence(log_probs)

perplexity = perplexity_object.compute_perplexity()

print(f"LSTM Model Perplexity : {perplexity:.2f}")

LSTM Model Perplexity : 756.53
